# EDA: UPI Digital Payments Analysis

This notebook uses numpy, pandas and matplotlib to explore the CSV files in the `data/` folder. It constructs proper quarterly datetimes from `Year`+`Quarter` columns when present and produces time series and top-state charts.

In [4]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

DATA_DIR = os.path.join('..', 'data')
OUT_DIR = os.path.join('..', 'outputs', 'plots')
os.makedirs(OUT_DIR, exist_ok=True)

def quarter_end_date(year, quarter):
    # quarter: 1..4 -> month 3,6,9,12 -> use last day of month
    try:
        y = int(year)
        q = int(quarter)
        month = {1:3, 2:6, 3:9, 4:12}.get(q, 12)
        return pd.Timestamp(year=y, month=month, day=1) + pd.offsets.MonthEnd(0)
    except Exception:
        return pd.NaT

def load_and_prepare(path):
    df = pd.read_csv(path)
    # If Year and Quarter present, create a 'date' column at quarter end
    if 'Year' in df.columns and 'Quarter' in df.columns:
        df['date'] = df.apply(lambda r: quarter_end_date(r['Year'], r['Quarter']), axis=1)
    else:
        # try to find a date-like column
        date_cols = [c for c in df.columns if 'date' in c.lower() or 'month' in c.lower() or 'year' in c.lower()]
        if date_cols:
            for c in date_cols:
                try:
                    df[c+'_parsed'] = pd.to_datetime(df[c], dayfirst=True, errors='coerce')
                except Exception:
                    df[c+'_parsed'] = pd.NaT
            # pick first parsed
            parsed = [c for c in df.columns if c.endswith('_parsed')]
            if parsed:
                df['date'] = df[parsed[0]]
    return df

# Example: load agg_trans.csv and plot
example = os.path.join(DATA_DIR, 'agg_trans.csv')
if os.path.exists(example):
    df = load_and_prepare(example)
    display(df.head())

    # aggregate monthly/quarterly time series
    if 'date' in df.columns and df['date'].notna().sum()>0:
        ts = df.set_index('date').resample('Q').sum(numeric_only=True)
        plt.figure(figsize=(10,4))
        if 'Transaction_count' in ts.columns:
            plt.plot(ts.index, ts['Transaction_count'], marker='o')
            plt.title('Quarterly Transaction Count (agg_trans)')
            plt.ylabel('Transaction_count')
            plt.grid(True)
            out = os.path.join(OUT_DIR, 'agg_trans_quarterly_transactions.png')
            plt.tight_layout()
            plt.savefig(out)
            print('Saved', out)
            plt.show()

    # Top 10 states by total transactions (across the whole file)
    if 'State' in df.columns and 'Transaction_count' in df.columns:
        grp = df.groupby('State', as_index=False)['Transaction_count'].sum().sort_values('Transaction_count', ascending=False).head(10)
        plt.figure(figsize=(10,5))
        plt.bar(grp['State'].astype(str), grp['Transaction_count'])
        plt.xticks(rotation=45, ha='right')
        plt.title('Top 10 States by Transaction Count (agg_trans)')
        out = os.path.join(OUT_DIR, 'agg_trans_top10_states.png')
        plt.tight_layout()
        plt.savefig(out)
        print('Saved', out)
        plt.show()
else:
    print('Example file not found:', example)

ModuleNotFoundError: No module named 'numpy'

In [5]:
# Analysis by Transaction Type and by State
# This cell groups the `agg_trans.csv` data by Transaction_type and State,
# plots quarterly time series per transaction type (top types) and top-10 states per type.
import matplotlib.pyplot as plt
import numpy as np

example = os.path.join(DATA_DIR, 'agg_trans.csv')
if not os.path.exists(example):
    print('agg_trans.csv not found in data folder')
else:
    df = load_and_prepare(example)
    # Ensure necessary cols exist
    if 'Transaction_type' not in df.columns or 'State' not in df.columns:
        print('Required columns (Transaction_type, State) not found in agg_trans.csv')
    else:
        # Group by date and transaction type
        grp = df.groupby(['date', 'Transaction_type'], as_index=False).agg({'Transaction_count': 'sum', 'Transaction_amount': 'sum'})
        pivot_count = grp.pivot(index='date', columns='Transaction_type', values='Transaction_count').fillna(0)
        # Pick top transaction types by total count
        top_types = pivot_count.sum().sort_values(ascending=False).head(6).index.tolist()

        # Plot time series for top types
        plt.figure(figsize=(12,6))
        for t in top_types:
            plt.plot(pivot_count.index, pivot_count[t], marker='o', label=str(t))
        plt.legend()
        plt.title('Quarterly Transaction Count by Transaction Type (top types)')
        plt.ylabel('Transaction_count')
        plt.xlabel('Date')
        plt.grid(True)
        out_ts = os.path.join(OUT_DIR, 'by_type_quarterly_transactions.png')
        plt.tight_layout()
        plt.savefig(out_ts)
        plt.show()
        print('Saved time series by type to', out_ts)

        # For each top type, compute top 10 states overall and for latest quarter
        latest_date = df['date'].max()
        for t in top_types:
            sub = df[df['Transaction_type'] == t]
            # overall top states
            overall = sub.groupby('State', as_index=False)['Transaction_count'].sum().sort_values('Transaction_count', ascending=False).head(10)
            plt.figure(figsize=(10,5))
            plt.bar(overall['State'].astype(str), overall['Transaction_count'])
            plt.xticks(rotation=45, ha='right')
            plt.title(f'Top 10 States by {t} (overall)')
            out_over = os.path.join(OUT_DIR, f'top10_states_{str(t).replace(' ', '_')}_overall.png')
            plt.tight_layout()
            plt.savefig(out_over)
            plt.show()
            print('Saved', out_over)

            # latest quarter top states
            latest_sub = sub[sub['date'] == latest_date] if pd.notna(latest_date) else sub
            if not latest_sub.empty:
                latest_states = latest_sub.groupby('State', as_index=False)['Transaction_count'].sum().sort_values('Transaction_count', ascending=False).head(10)
                plt.figure(figsize=(10,5))
                plt.bar(latest_states['State'].astype(str), latest_states['Transaction_count'])
                plt.xticks(rotation=45, ha='right')
                plt.title(f'Top 10 States by {t} (latest quarter)')
                out_lat = os.path.join(OUT_DIR, f'top10_states_{str(t).replace(' ', '_')}_latest.png')
                plt.tight_layout()
                plt.savefig(out_lat)
                plt.show()
                print('Saved', out_lat)
            else:
                print('No data for latest quarter for type', t)

SyntaxError: f-string: unmatched '(' (1270409877.py, line 47)